In [1]:
from sqlalchemy.ext.asyncio import create_async_engine, async_sessionmaker
from sqlalchemy.orm import DeclarativeBase
from src.config import config
from src.database import Base, AsyncSessionLocal

In [2]:
from fastapi_users.db import SQLAlchemyBaseUserTable
from sqlalchemy import Boolean, Integer, String, BigInteger, ForeignKey, Table, Column
from sqlalchemy.orm import Mapped, mapped_column, relationship


class User(SQLAlchemyBaseUserTable[int], Base):
    __tablename__ = "User"
    id: Mapped[int] = mapped_column(Integer, primary_key=True, index=True)
    name: Mapped[str] = mapped_column(String(length=32), nullable=False)
    surname: Mapped[str] = mapped_column(String(length=32), nullable=False)
    patronymic: Mapped[str] = mapped_column(String(length=32), nullable=False)
    email: Mapped[str] = mapped_column(String(length=320), unique=True, index=True, nullable=False)
    phone_number: Mapped[str] = mapped_column(String(length=11), nullable=False)
    hashed_password: Mapped[str] = mapped_column(String(length=1024), nullable=False)
    games_played: Mapped[int] = mapped_column(Integer, nullable=False, default=0)
    win_games: Mapped[int] = mapped_column(Integer, nullable=False, default=0)
    # FastAPI-Users defaults:
    is_active: Mapped[bool] = mapped_column(Boolean, default=True, nullable=False)
    is_superuser: Mapped[bool] = mapped_column(Boolean, default=False, nullable=False)
    is_verified: Mapped[bool] = mapped_column(Boolean, default=False, nullable=False)
    
association_table = Table(
    "TeamUsers",
    Base.metadata,
    Column("team_id", ForeignKey("Team.id")),
    Column("user_id", ForeignKey("User.id")),
)
    
class Team(Base):
    __tablename__ = "Team"
    id: Mapped[int] = mapped_column(Integer, primary_key=True, index=True)
    owner_id: Mapped[int] = mapped_column(ForeignKey("User.id"), nullable=True)
    name: Mapped[str] = mapped_column(String(length=32), nullable=False)
    games_played: Mapped[int] = mapped_column(Integer, nullable=False, default=0)
    win_games: Mapped[int] = mapped_column(Integer, nullable=False, default=0)
    users: Mapped[list[User]] = relationship(lazy="raise", secondary=association_table)


In [5]:
from sqlalchemy import select, exists, func
from sqlalchemy.orm import selectinload

async def test():
    async with AsyncSessionLocal() as db:
        query = select(Team).filter(Team.id == 1).options(selectinload(Team.users))
        result = await db.execute(query)
        team = result.scalars().first()
        
        query = select(User).filter(User.id == 1)
        result = await db.execute(query)
        user = result.scalars().first()
        
        team.users.append(user)
        
        print(team.users)
        
        await db.commit()
        
await test()

2023-04-30 14:56:33,027 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-30 14:56:33,028 INFO sqlalchemy.engine.Engine SELECT "Team".id, "Team".owner_id, "Team".name, "Team".games_played, "Team".win_games 
FROM "Team" 
WHERE "Team".id = $1::INTEGER
2023-04-30 14:56:33,029 INFO sqlalchemy.engine.Engine [cached since 26.71s ago] (1,)
2023-04-30 14:56:33,034 INFO sqlalchemy.engine.Engine SELECT "Team_1".id AS "Team_1_id", "User".id AS "User_id", "User".name AS "User_name", "User".surname AS "User_surname", "User".patronymic AS "User_patronymic", "User".email AS "User_email", "User".phone_number AS "User_phone_number", "User".hashed_password AS "User_hashed_password", "User".games_played AS "User_games_played", "User".win_games AS "User_win_games", "User".is_active AS "User_is_active", "User".is_superuser AS "User_is_superuser", "User".is_verified AS "User_is_verified" 
FROM "Team" AS "Team_1" JOIN "TeamUsers" AS "TeamUsers_1" ON "Team_1".id = "TeamUsers_1".team_id JOIN "User" ON "Us

In [ ]:
await engine.dispose()